In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision 
import pandas as pd

In [2]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\dataset\covid.train.csv',)
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\dataset\covid.test.csv')

In [3]:
train_data.shape

(2700, 95)

In [4]:
test_data.shape

(893, 94)

In [5]:
all_features=pd.concat((train_data.iloc[:,1:-1],test_data.iloc[:,1:]))
all_features.shape

(3593, 93)

In [6]:
all_features.isnull().sum().sum()

0

In [7]:
all_features.iloc[:,40:]=all_features.iloc[:,40:].apply(lambda x: (x-x.mean()) / x.std())

In [8]:
all_features.shape

(3593, 93)

In [30]:
all_features.iloc[:5,[57,75]]

,tested_positive,tested_positive.1
0,0.438190,0.505859
1,0.511783,0.577989
2,0.583781,0.654668
3,0.660320,0.638203
4,0.643885,0.472574


In [65]:
# n_train=train_data.shape[0]
# train_features=torch.Tensor(all_features.iloc[:n_train,:].values)
# test_features=torch.Tensor(all_features.iloc[n_train:,:].values)
# train_labels=torch.Tensor(train_data.iloc[:,-1].values).view(-1,1)

#2 features

n_train=train_data.shape[0]
train_features=torch.Tensor(all_features.iloc[:n_train,[57,75]].values)
test_features=torch.Tensor(all_features.iloc[n_train:,[57,75]].values)
train_labels=torch.Tensor(train_data.iloc[:,-1].values).view(-1,1)


In [66]:
loss=nn.MSELoss()

In [67]:
def get_net(num_features):
#     net=nn.Linear(num_features,1)
    net = nn.Sequential(nn.Linear(num_features, 186), 
                        nn.ReLU(),
                        nn.Linear(186, 64), 
                        nn.ReLU(),
                        nn.Linear(64, 1))
    for p in net.parameters():
        nn.init.normal_(p, mean=0, std=0.01)
    return net

In [68]:
def rmse(net,features,labels):
    net.eval()
    with torch.no_grad():
        rmse=torch.sqrt(loss(net(features),labels).mean()).item()
    net.train()
    return rmse

In [69]:
def train(net,train_features,train_labels,test_features,test_labels,
         num_epochs,learning_rate,weight_decay,batch_size):
    train_ls,test_ls=[],[]
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    train_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    optimizer=torch.optim.Adam(net.parameters(),lr=learning_rate,weight_decay=weight_decay)
    min_rmse=1000.
    early_stop_count=300
    for e in range(num_epochs):
        i=0
        for x,y in train_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()

        
        train_ls.append(rmse(net,train_features,train_labels))
        if test_labels is not None:
            test_ls.append(rmse(net,test_features,test_labels))
            dev_rmse=test_ls[-1]    
            if dev_rmse<min_rmse:
                min_rmse=dev_rmse
                print('save model (epoch %d) train rmse %f, test rmse %f ' %(
                e+1,train_ls[-1],min_rmse))
                torch.save(net.state_dict(),r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\models\model.pht')
                i=0
            else:
                i+=1
            if i>early_stop_count:
                break
            
                
                
        
    return train_ls,test_ls

In [70]:
def get_k_fold_data(k,i,x,y):
    assert k>1
    fold_size=x.shape[0]//k
    x_train,y_train=None,None
    index_list=list(range(x.shape[0]))
    np.random.shuffle(index_list)
    for j in range(k):
        idx=slice(j*fold_size,(j+1)*fold_size)
        x_part,y_part=x[index_list[idx],:],y[index_list[idx]]
        
        if j==i:
            x_valid,y_valid=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    
    return x_train,y_train,x_valid,y_valid

In [71]:
def get_net(num_features):
#     net=nn.Linear(num_features,1)
    net = nn.Sequential(nn.Linear(num_features, 64), 
#                         nn.ReLU(),
#                         nn.Linear(128, 64), 
                        nn.ReLU()
                        nn.Linear(64, 1))
    for p in net.parameters():
        nn.init.normal_(p, mean=0, std=0.01)
    return net

In [72]:
def k_fold(k,x_train,y_train,num_epochs,learning_rate,weight_decay,batch_size):
    train_l_sum,valid_l_sum=0,0
    for i in range(k):
        data=get_k_fold_data(k,i,x_train,y_train)
        net=get_net(x_train.shape[1])
        train_ls,valid_ls=train(net,*data,num_epochs,learning_rate,
                           weight_decay,batch_size)
        train_l_sum+=train_ls[-1]
        valid_l_sum+=valid_ls[-1]
        
        print('fold %d,train rmse %f,valid %f' % (
            i,train_ls[-1],valid_ls[-1]))
    return train_l_sum/k,valid_l_sum/k
        

In [73]:
k, num_epochs, lr, weight_decay, batch_size = 5, 100, 0.05, 0.5, 800
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                          weight_decay, batch_size)
print('\n %d-fold validation: avg train rmse %f, avg valid rmse %f' %
(k, train_l, valid_l))

save model (epoch 1) train rmse 16.955643, test rmse 17.220272 
save model (epoch 2) train rmse 13.654101, test rmse 13.843808 
save model (epoch 3) train rmse 9.009602, test rmse 8.905493 
save model (epoch 4) train rmse 4.629296, test rmse 4.402546 
save model (epoch 7) train rmse 3.576016, test rmse 3.500607 
save model (epoch 10) train rmse 2.892408, test rmse 2.715712 
save model (epoch 11) train rmse 1.709682, test rmse 1.565934 
save model (epoch 15) train rmse 1.467478, test rmse 1.478957 
save model (epoch 16) train rmse 1.309696, test rmse 1.304996 
save model (epoch 18) train rmse 1.160902, test rmse 1.157113 
save model (epoch 22) train rmse 1.150191, test rmse 1.148911 
save model (epoch 30) train rmse 1.117253, test rmse 1.140632 
save model (epoch 31) train rmse 1.118838, test rmse 1.138116 
save model (epoch 35) train rmse 1.110472, test rmse 1.134955 
save model (epoch 39) train rmse 1.102780, test rmse 1.128901 
save model (epoch 48) train rmse 1.096506, test rmse 1.1

In [21]:
class NeuralNet(nn.Module):
    ''' A simple fully-connected deep neural network '''
    def __init__(self, input_dim):
        super(NeuralNet, self).__init__()

        # Define your neural network here
        # TODO: How to modify this model to achieve better performance?
        self.net = nn.Sequential(
#             nn.BatchNorm1d(input_dim),
#             nn.Linear(input_dim, 86),
#             nn.ReLU(),
# #             nn.BatchNorm1d(64),
#             nn.Linear(86, 1)
            nn.Linear(input_dim,1)
        )

        # Mean squared error loss
        self.criterion = nn.MSELoss(reduction='mean')

    def forward(self, x):
        ''' Given input of size (batch_size x input_dim), compute output of the network '''
        return self.net(x).squeeze(1)

    def cal_loss(self, pred, target):
        ''' Calculate loss '''
        # TODO: you may implement L2 regularization here
        return self.criterion(pred, target)

In [33]:
model=model = NeuralNet(93)
model_chk=torch.load(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\models\model.pth')
model.load_state_dict(model_chk)

train_mse=rmse(model,train_features,train_labels)
print(train_mse)

15.147257804870605


In [34]:
model.eval()
((model(train_features).view(-1,1).detach()-train_labels)**2).mean().item()

1.4321184158325195

In [ ]:
model.eval()
((model(train_features).view(-1,1).detach()-train_labels)**2).mean().item()

In [60]:
loss=nn.MSELoss()
model.eval()
loss(model(train_features).view(-1,1).detach() , train_labels).mean().item()

1.4321180582046509

In [62]:
train_features[:5,40:]

tensor([[-0.4117, -0.5661, -0.4015, -0.3501, -0.9618,  1.3095,  0.9901,  1.7442,
          1.2648,  1.2836,  1.4166, -0.6311, -1.1444, -0.6191, -0.8205, -1.8231,
         -0.2361,  0.4382, -0.3616, -0.4875, -0.4080, -0.3551, -1.0991,  1.3260,
          1.0272,  1.8005,  1.2503,  1.2497,  1.3251, -0.6742, -1.2312, -0.6082,
         -0.9910, -1.8735, -0.1731,  0.5059, -0.2294, -0.3054, -0.3758, -0.3375,
         -1.0290,  1.2156,  0.9943,  1.6239,  1.2857,  1.1208,  1.2708, -0.7290,
         -1.1782, -0.5782, -0.9398, -1.7712, -0.1820],
        [-0.3534, -0.4798, -0.3982, -0.3456, -1.0841,  1.3110,  1.0142,  1.7842,
          1.2391,  1.2316,  1.3127, -0.6792, -1.2313, -0.6141, -0.9865, -1.8530,
         -0.1689,  0.5118, -0.2211, -0.2978, -0.3665, -0.3290, -1.0159,  1.2019,
          0.9844,  1.6035,  1.2721,  1.1039,  1.2580, -0.7323, -1.1820, -0.5856,
         -0.9354, -1.7531, -0.1767,  0.5780, -0.0501, -0.1217, -0.4092, -0.3946,
         -1.0876,  1.0830,  0.9917,  1.5324,  1.2628, 

In [36]:
net=get_net(train_features.shape[1])
net_chk=torch.load(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\models\model.wx')
net.load_state_dict(net_chk)

In [37]:
net.eval()
((net(train_features).view(-1,1).detach()-train_labels)**2).mean().item()

1.2438639714673627e-05

In [55]:
net.eval()
test_pred=net(test_features).view(-1,1).detach().numpy()
test_data['tested_positive']=test_pred.reshape(1,-1)[0]
submission=pd.concat([test_data['id'],test_data['tested_positive']],axis=1)
submission
test_pred.shape

(893, 1)

In [58]:
submission.to_csv(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\dataset\prednew68.csv',
                 index=False)